# TensorflowによるRNNの実装
<div style="text-align: center;">
〜RNNを用いた言語モデルの学習〜
</div>


言語モデルとは：

ある単語列が与えられたときに次にどんな単語が来るかを予測する確率モデル

<img src="./fig/language_model.png"　 width="300" >

##  目次
- [目標](#目標)
- [下準備](#下準備)
- [言語モデルの実装](#言語モデルの実装)
- [実行](#実行)

<a name="目標"></a>
### 目標
- RNNを実装する

<a name="下準備"></a>
### 下準備

#### Tensorflowのインストール

In [ ]:
!sudo pip install tensorflow-gpu==1.2

#### ライブラリのインポート

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import time
import numpy as np
import tensorflow as tf
import tool

In [ ]:
data_path = './data/'
save_path = './model/' 

#### ハイパーパラメータの設定
- init_scale：重みの初期化の範囲
- fix_epoch：学習率を固定するエポック数
- max_epoch：エポック数
- vocab_size：語彙数
- learning_rate：学習率
- hidden_size：LSTMの隠れ層の次元数
- num_steps：LSTMに入力する文章の長さ
- batch_size：バッチサイズ

In [ ]:
init_scale = 0.1
fix_epoch = 10
max_epoch = 20
learning_rate = 0.1
hidden_size = 200
num_steps = 20
batch_size = 20  

#### データの読み込み
- train_data：学習データ
- valid_data：評価用データ
- test_data：テストデータ
- word_to_id：単語（キー）とid（値）の辞書
- vocab_size：単語数

In [ ]:
train_data, valid_data, test_data, word_to_id = tool.ptb_raw_data(data_path)
vocab_size = len(word_to_id) #単語数

#### データ例
< eos > : end of sentence(文末)を表す

In [ ]:
print(' '.join(tool.ids_to_words(train_data[164:197], word_to_id)))

<a name="言語モデルの実装"></a>
### 言語モデルの実装

#### モデルの概要　
<img src="./fig/model.png"　 width="600" >

####  1：入力→分散表現
Tensorflowで単語のIDから分散表現に直すには次の２つの関数を使用します．
- tf.Variable
- tf.nn.embedding_lookup
<br />
<br />

1-1：各行が各単語の分散表現に対応すテンソル"embedding"を初期化します．
<div style="text-align: center;">
```python 
embedding = tf.Variable(tf.random_uniform([vocab_size, hidden_size], -init_scale, init_scale))
```
</div>
- 初期化には`tf.random_uniform(shape, minval, maxval)`を使います．
    - 最小値：minval, 最大値：maxvalの連続一様分布を使ってshape：shapeのテンソルを初期化することができます．
    - shapeは[単語の総数, LSTMの隠れ層の次元]を指定します．
    - minvalとmaxvalにはハイパーパラメータで設定した-init_scaleとinit_scaleを代入します 
<br />
<br />

1-2：embeddingからinput_dataに対応する分散表現を求めます．
<div style="text-align: center;">
```python
tf.nn.embedding_lookup(embeddings, input_data)
```
</div>
- `tf.nn.embedding_lookup(params, ids)`ではテンソルparamsからidsに含まれる要素を調べることができます．
    - `tf.nn.embedding_lookup`の使用例：
```python
>>> params = tf.constant([[1,2,3], [4,5,6], [7,8,9]])
>>> ids = tf.constant([2, 0])
>>> word_embedding = tf.nn.embedding_lookup(params, ids)
>>> with tf.Session() as sess:
・・・      print(sess.run(word_embedding))
[[7 8 9]
 [1 2 3]]
```

#### 2：分散表現→LSTM→LSTMの出力
分散表現をLSTMに入力しLSTMの出力を得ます．

2-1：RNNセルの初期化
<div style="text-align: center;">
```python
self.cell = tf.contrib.rnn.BasicLSTMCell(hidden_size)
```
</div>
- Tensorflowには様々なRNNが定義されていますが，ここでは`tf.contrib.rnn.BasicLSTM`を使用します．
- tf.contrib.rnn.BasicLSTM(num_units)でユニット数を指定します．

2-2：LSTMの内部状態の初期化
<div style="text-align: center;">
```python
self.initial_state = cell.zeros([self.batch_size, hidden_size * 2], tf.float32)
```
</div>
- LSTMは短期記憶hと長期記憶cを所持しています．（講義資料参照）
- 短期記憶hと長期記憶cを0ベクトルで初期化します．

2-3：RNNグラフの構築
- `tf.nn.dynamic_rnn`により，2-1で定義したcellを用いてRNNを構築します．
- `tf.nn.dynamic_rnn`の主な入力は以下のようです．
```python
tf.nn.dynamic_rnn(
    cell,
    inputs,
    initial_state=None,
    dtype=None
)
```
- 返り値としてlstm_outputとstateを受け取ります．

#### 3：LSTMの出力→Dense Layer→予測
LSTMの出力に線形射影を用いて予測結果を計算します．

3-1：shapeの変更
- LSTMの出力lstm_outputは[batch_size, num_steps, hidden_size]のshapeです．
- Dense Layerの入力に適するようにtf.reshapeを用いてshapeを変更しましょう．

3-2：Dense Layerの構築
- tf.layers.denseを用いてDense Layerを構築します．
- `tf.layers.dense`の主な入力は以下のようです．
```python
tf.layers.dense(
    inputs,
    units
)
```

In [14]:
class Model(object):
    def __init__(self):
        self.input_data = tf.placeholder(tf.int32, [None, None])
        batch_size = tf.shape(self.input_data)[0]
        num_steps = tf.shape(self.input_data)[1]
        self.target = tf.placeholder(tf.int32, [None, None])
        
        # TODO1：入力→分散表現
        embedding = tf.Variable(tf.random_uniform([vocab_size, hidden_size], -init_scale, init_scale)) #初期化
        emb_data = tf.nn.embedding_lookup(embedding, self.input_data) #分散表現に変換
        # TODO2：分散表現→LSTM→LSTMの出力
        cell = tf.contrib.rnn.BasicLSTMCell(hidden_size) #RNNセルの初期化
        self.initial_state = cell.zero_state(batch_size, tf.float32) #LSTMの内部状態の初期化
        lstm_output, state = tf.nn.dynamic_rnn(cell, emb_data, initial_state=self.initial_state, dtype=tf.float32) #RNNグラフの構築
        # TODO3：LSTMの出力→Dense Layer→予測
        lstm_output = tf.reshape(lstm_output, [-1, hidden_size]) #shapeの変更
        prediction = tf.layers.dense(lstm_output, vocab_size) #Dense Layerの構築
        
        # sequenctial ロスを計算するために三次元テンソルに変換する
        prediction = tf.reshape(prediction, [batch_size, num_steps, vocab_size])
        #ロスの計算
        loss = tf.contrib.seq2seq.sequence_loss(prediction, self.target, 
            tf.ones([batch_size, num_steps], dtype=tf.float32),
            average_across_timesteps=False)

        # コスト（ロスの合計）を更新
        self.output = prediction
        self.cost = tf.reduce_sum(loss)
        self.final_state = state

        #最適化手法の設定
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.cost)

### 実行
<a name="実行"></a>

#### データの整形とモデルの生成

In [ ]:
train_input = tool.Input(batch_size, num_steps, data=train_data)
valid_input = tool.Input(batch_size, num_steps, data=valid_data)
model = Model()

#### 1epochの挙動

In [ ]:
def run_epoch(session, model, input_data, train=False, verbose=False):
    """Runs the model on the given data."""
    start_time = time.time()
    costs = 0.0
    iters = 0

    fetches = {"cost": model.cost, "final_state": model.final_state,}
    if train:
        fetches["optimizer"] = model.optimizer

    for step in range(input_data.epoch_size):
        feed_dict = {}
        feed_dict[model.input_data], feed_dict[model.target] = input_data.next()
        if step>0:
            feed_dict[model.initial_state] = state
        vals = session.run(fetches, feed_dict)
        cost = vals["cost"]
        state = vals["final_state"]

        costs += cost
        iters += input_data.num_steps

        if verbose and step % (input_data.epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                (step * 1.0 / input_data.epoch_size, np.exp(costs / iters),
                 iters * input_data.batch_size / (time.time() - start_time)))

    return np.exp(costs / iters)

#### 学習

In [ ]:
saver = tf.train.Saver()
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    for i in range(max_epoch):
        print("Epoch: %d" % (i + 1))
        train_perplexity = run_epoch(session, model, train_input, train=True, verbose=True)
        print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
        valid_perplexity = run_epoch(session, model, valid_input)
        print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

    # モデルの保存
    print("Saving model to %s" % save_path +"model.ckpt")
    saver.save(session, save_path)

#### テスト

実際にモデルに単語を入力して，予測される続きの文を表示してみましょう．

2つ下のセルにあるwordsを編集してセルを実行してみましょう．

In [ ]:
def generate(session, model, state, test_input):
    fetches = {"output": model.output, "final_state": model.final_state,}
    feed_dict = {}
    if state:
        feed_dict[model.initial_state] = state
    feed_dict[model.input_data] = test_input
    vals = session.run(fetches, feed_dict)
    prediction = vals["output"]
    state = vals["final_state"]
    prediction[0,0,1] = 0
    return np.argmax(prediction), state

In [ ]:
words = "I am" # これを編集
input_words = [word_to_id[word] for word in words.lower().split(" ") if word in word_to_id]
gen_max = 20

In [ ]:
with tf.Session() as session:
    generate_words = []
    saver.restore(session, save_path)
    state=None
    for input_ in input_words:
        pred_word, state= generate(session, model, state, np.array([[input_]]))
    generate_words.append(tool.ids_to_words([pred_word], word_to_id)[0])
    for i in range(gen_max):
        if pred_word == word_to_id["<eos>"]:
            break
        pred_word, state = generate(session, model, state, np.array([[pred_word]]))
        generate_words.append(tool.ids_to_words([pred_word], word_to_id)[0])

print("prediction:")
print(words + " " + "  ".join(generate_words))